# RatU Day5OpenfieldSD recording info

In [1]:
import numpy as np
import subjects

sess = subjects.sd.ratUday5[0] # add your subject session
print(sess.recinfo)

filename: /data/Clustering/sessions/RatU/RatUDay5OpenfieldSD/RatU_Day5OpenfieldSD_2021-08-04_08-44-31.xml 
# channels: 192
sampling rate: 30000
lfp Srate (downsampled): 1250


# Animal metadata

In [2]:
from neuropy.core import animal

d = {'name':'hello','tag':'ser'}

an = animal.Animal(d) 


# Probe configuration
- RatU_Day5OpenfieldSD has two probes: both 128chan-8shanks diagnostic biochips.
- 64 channels in one of the probes (implanted in left hemisphere) had no signal from 4 shanks (probably one of the intan chips was faulty). So only 192 channels were recorded, the channels in .dat file are already order according to depth.


In [ ]:
%matplotlib inline
from neuropy.core import shank, probe, probegroup
from neuropy.plotting import plot_probe

shanks = []
channel_groups = sess.recinfo.channel_groups
badchans = sess.recinfo.skipped_channels

#--- diagnostic-biochip 8 shanks -----------
for i in range(8):
    chans = channel_groups[i]
    shank = shank.auto_generate(
        columns=2,
        contacts_per_column=8,
        xpitch=15,
        ypitch=15,
        y_shift_per_column=[0,-7.5],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)
    shanks.append(shank)

probe1 = probe(shanks)

#--- dignostic biochip 4 shanks (4 shanks were bad) ----------- 
shanks = []
for i in range(8,12):

    shank = shank.auto_generate(
        columns=2,
        contacts_per_column=8,
        xpitch=15,
        ypitch=15,
        y_shift_per_column=[0, -7.5],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)
    shanks.append(shank)


probe2 = probe(shanks)
probe2.move((probe1.x_max+500,0))

prbgrp = probegroup()
prbgrp.add_probe(probe1)
prbgrp.add_probe(probe2)

prbgrp.filename = sess.fileprefix.with_suffix(".probegroup.npy")
prbgrp.save()
plot_probe(prbgrp)

# Experimental paradigm
- Openfield paradigm was used
- Following opnefield arena alternation, animal was sleep deprived for 2 h.

In [4]:
sess.paradigm.to_dataframe()

,start,stop,label,duration
0,0,7124,pre,7124
1,7125,11745,maze,4620
2,9591,11745,sprinkle,2154
3,11746,18957,sd,7211
5,11746,37933,post,26187
4,18958,37933,rs,18975


In [12]:
import pandas as pd
from neuropy.core import Epoch

epochs = pd.DataFrame(
    {
        "start": [0,7125,9591,11746,18958,11746],
        "stop": [7124,11745,11745,18957,37933,37933],
        "label": ["pre", "maze",'sprinkle', "sd",'rs','post'],
    }
)

paradigm= Epoch(epochs=epochs)
paradigm.filename = sess.filePrefix.with_suffix(".paradigm.npy")
paradigm.save()

RatU_Day5OpenfieldSD_2021-08-04_08-44-31.paradigm.npy saved


# Neurons
- import spiketrains from Phy
- estimate neuron types such pyramidal, interneuron etc. 

## Importing spiketrains from Phy

In [5]:
from neuropy.io import PhyIO
from neuropy.core import Neurons
from pathlib import Path
import numpy as np

cluster_path = Path(
    "/home/bapung/Documents/ClusteringHub/spykcirc/RatU/RatUDay5OpenfieldSD/RatU_Day5OpenfieldSD_2021-08-04_08-44-31-1.GUI"
)
chan_grps = sess.recinfo.channel_groups
phy_data = PhyIO(cluster_path)
spiketrains = phy_data.spiketrains
peak_chans = phy_data.peak_channels
waveforms = phy_data.peak_waveforms
shank_id = sess.probegroup.get_shank_id_for_channels(peak_chans)

neuron_type_id = phy_data.cluster_info.q.values
neuron_type = np.ones(len(neuron_type_id), dtype="U5")
neuron_type[neuron_type_id<4] = 'pyr'
neuron_type[neuron_type_id==6] = 'mua'
neuron_type[neuron_type_id==8] = 'inter'

neurons = Neurons(
    np.array(spiketrains, dtype=object),
    t_stop=sess.eegfile.duration,
    sampling_rate=phy_data.sampling_rate,
    peak_channels=peak_chans,
    waveforms=np.array(waveforms, dtype="object"),
    shank_ids=np.array(shank_id).astype(int),
    neuron_type=neuron_type
)

neurons.filename = sess.filePrefix.with_suffix(".neurons")
neurons.save()


In [7]:
neurons.save()

RatU_Day5OpenfieldSD_2021-08-04_08-44-31.neurons saved


### Plot raster

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from neuropy.plotting import plot_raster

plot_raster(neurons,color='jet',add_vert_jitter=True)

## BinnedSpiketrain and Mua objects using Neurons

In [ ]:
mua =sess.neurons.get_mua()
mua.filename = sess.filePrefix.with_suffix(".mua.npy")
mua.save()   


In [ ]:
%matplotlib widget
from neuropy import plotting
smth_mua = sess.mua.get_smoothed(sigma=0.02)
plotting.plot_mua(smth_mua)

# Detect epochs
Here we will various types of epochs which typical for hippocampal recordings.

## Artifacts
A typical session will have some artifacts that may negatively influence many analyses. Using a simple zscore measure, we can identify epochs where signal is above some threshold.

In [7]:
%matplotlib widget
from neuropy.analyses import detect_artifact_epochs
from neuropy import plotting
from neuropy.utils import signal_process
from neuropy.core import Signal
print(sess)
signal = sess.eegfile.get_signal([63])
filt_trace = signal_process.filter_sig.highpass(signal.traces,cutoff=400)
filtered_signal = Signal(filt_trace,signal.sampling_rate)
artifact_epochs = detect_artifact_epochs(filtered_signal, thresh=20)
artifact_epochs.filename = sess.filePrefix.with_suffix(".artifact.npy")
artifact_epochs.save()
# plotting.plot_artifact_epochs(artifact_epochs, filtered_signal)


ProcessData(RatU_Day5OpenfieldSD_2021-08-04_08-44-31.xml)
RatU_Day5OpenfieldSD_2021-08-04_08-44-31.artifact.npy saved


### write artifact epochs to spyking circus format

In [10]:
sess.artifact.to_dataframe()

,start,stop,label,duration
0,10420.7032,10476.6904,,55.9872
1,13627.6792,13629.5864,,1.9072
2,14444.2480,14444.2616,,0.0136
3,14605.6816,14605.6952,,0.0136
4,17042.4912,17042.9944,,0.5032
5,18076.9688,18077.1992,,0.2304
6,18125.1592,18125.1760,,0.0168
7,18511.6568,18512.1632,,0.5064
8,33856.3104,33907.1072,,50.7968
9,34233.8752,34237.9208,,4.0456


In [9]:
from neuropy.io import SpykingCircusIO

file = sess.filePrefix.with_suffix('.dead')
SpykingCircusIO.write_epochs(file,sess.artifact)

RatU_Day5OpenfieldSD_2021-08-04_08-44-31.dead created


## Sleep scoring

In [6]:
from neuropy.core import Signal
from neuropy.analyses import brainstates

signal = sess.eegfile.get_signal() 
brainstates = brainstates.detect_brainstates_epochs(signal=signal,probe=sess.probegroup)

starting emg calculation
emg calculation done
channel for sleep detection: 113
spectral properties calculated


In [8]:
brainstates.filename = sess.filePrefix.with_suffix('.brainstates')
brainstates.save()

RatU_Day5OpenfieldSD_2021-08-04_08-44-31.brainstates saved


## Ripple
To detect ripples one also needs probegroup.

In [ ]:
from neuropy.analyses import oscillations
signal = sess.eegfile.get_signal()
ripple_epochs =oscillations.detect_ripple_epochs(signal, sess.probegroup)
ripple_epochs.filename = sess.filePrefix.with_suffix('.ripple.npy')
ripple_epochs.save()

## Pbe

In [ ]:
from neuropy.analyses import detect_pbe_epochs

pbe = detect_pbe_epochs(smth_mua)
pbe.filename = sess.filePrefix.with_suffix('.pbe')
pbe.save()


# Assign position data

In [ ]:
from neuropy.io import OptitrackIO
from neuropy.core import Position
from pathlib import Path

opti_folder = sess.filePrefix.parent / 'position'
opti_data = OptitrackIO(dirname=opti_folder,scale_factor=0.25)

In [ ]:
import pandas as pd
from datetime import datetime

#---- startimes of concatenated .dat files
tracking_sRate = opti_data.sampling_rate
rec_datetime = pd.read_csv(sess.filePrefix.with_suffix('.datetime.csv'))
data_time = []
for i, file_time in enumerate(rec_datetime["StartTime"]):
    sync_time = rec_datetime['sync_nframes'][i]/rec_datetime['sync_rate'][i]
    tbegin = datetime.strptime(file_time, "%Y-%m-%d_%H-%M-%S") + pd.Timedelta(sync_time,unit='sec')
    nframes = rec_datetime["nFrames"][i]
    duration = pd.Timedelta(nframes / sess.recinfo.dat_sampling_rate, unit="sec")
    tend = tbegin + duration
    trange = pd.date_range(
        start=tbegin,
        end=tend,
        periods=int(duration.total_seconds() * tracking_sRate),
    )
    data_time.extend(trange)
data_time = pd.to_datetime(data_time)

# ------- deleting intervals that were deleted from .dat file after concatenating
ndeletedintervals = rec_datetime.count()["deletedStart (minutes)"]
for i in range(ndeletedintervals):
    tnoisy_begin = data_time[0] + pd.Timedelta(
        rec_datetime["deletedStart (minutes)"][i], unit="m"
    )
    tnoisy_end = data_time[0] + pd.Timedelta(
        rec_datetime["deletedEnd (minutes)"][i], unit="m"
    )

    del_index = np.where((data_time > tnoisy_begin) & (data_time < tnoisy_end))[
        0
    ]

    data_time = np.delete(data_time, del_index)

x,y,z = opti_data.get_position_at_datetimes(data_time)
traces = np.vstack((z,x,y))

position = Position(traces=traces,t_start=0,sampling_rate=opti_data.sampling_rate)
position.filename = sess.filePrefix.with_suffix('.position.npy')
position.save()

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

# plt.plot(opti_data.datetime_array,opti_data.z)
plt.plot(position.x,position.y)

In [ ]:
%matplotlib widget
from neuropy.utils import signal_process

signal = sess.eegfile.get_signal([40])

spec = signal_process.spectrogramBands(signal.traces[0])


In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt

plt.plot(spec.time,spec.theta/500)
plt.plot(sess.position.time,sess.position.y)
plt.plot(sess.position.time[1:],np.diff(sess.position.y)*100)
plt.xlim([12500,12600])
plt.ylim([-400,400])

## Linearize position

In [ ]:
from neuropy.utils import position_util

maze = sess.paradigm['maze']
maze_pos = sess.position.time_slice(maze[0],maze[1])
linear_pos = position_util.linearize_position(maze_pos)

In [ ]:
%matplotlib widget
plt.plot(maze_pos.time,maze_pos.x)
plt.plot(linear_pos.time,linear_pos.x)